In [1]:
import pandas as pd
import numpy as np
from os import listdir


import InfoTrayectosModule as Tracks
from DecoderModule import Decoder
import InfoStationsModule as Stations


import plotly.express as px
import plotly.graph_objects as go
mapbox_access_token = open("mapbox_token").read()

In [2]:
filePath= "../DataFrames/Stations/"
data = Stations.concatenateAll(filePath)

Decode = Decoder('../DataFrames/DatosEstaticos.txt')

1
['201907.pkl', '201908.pkl', '201909.pkl', '201910.pkl', '201911.pkl', '201912.pkl', '202001.pkl', '202002.pkl', '202003.pkl', '202004.pkl', '202005.pkl', '202006_stations.pkl', 'Bicimad_Estacions_201808.pkl', 'Bicimad_Estacions_201809.pkl', 'Bicimad_Estacions_201810.pkl', 'Bicimad_Stations_201811.pkl', 'Bicimad_Stations_201812.pkl', 'Bicimad_Stations_201901.pkl', 'Bicimad_Stations_201902.pkl', 'Bicimad_Stations_201903.pkl', 'Bicimad_Stations_201904.pkl', 'Bicimad_Stations_201905.pkl', 'Bicimad_Stations_201906.pkl']


In [3]:
StartDate = '2020-5-06 00:48'
EndDate = '2020-5-13 00:50'
Value='DockBikes'
Comparator = 'dow'
Accuracy = "d"





df = Stations.ChooseInterval(data, StartDate, EndDate, Value)
df = Stations.ChooseAccuracy(df, Accuracy)

df

ID,C,E,N,O,S
Date,,,,,
2020-05-06,9.354839,12.000000,11.317073,11.163636,11.578947
2020-05-07,9.645161,11.897959,10.756098,11.654545,10.947368
2020-05-08,9.612903,11.285714,12.024390,11.309091,10.447368
2020-05-09,9.451613,11.918367,11.536585,10.790909,10.210526
2020-05-10,8.483871,11.602041,10.524390,10.736364,10.092105
2020-05-11,8.774194,12.367347,13.207317,9.663636,9.302632
2020-05-12,9.322581,12.469388,11.719512,10.900000,11.131579
2020-05-13,9.451613,12.061224,10.731707,11.618182,11.973684


In [4]:
toPlot = Stations.Ploteo(df.iloc[0], Decode)


# Mapa estático

In [23]:

fig = go.Figure(go.Scattermapbox(
        lat=toPlot.Latitude,
        lon=toPlot.Longitude,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=toPlot.Value, color= toPlot.Value, showscale=True
        ),
        text=toPlot.Address,
       
        
    ))

fig.update_layout(
    title = dict( 
        text= toPlot.Date,
        x=0.5,
    ),

    autosize=True,
    hovermode='closest',
    width=750,
    height=500,
    margin=dict(
        l=50,
        r=0,
        b=50,
        t=50
        
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=float(toPlot.Latitude[0]),
            lon= float(toPlot.Longitude[0])
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()

# Mapa con slider

In [33]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in range(0,len(df)):
   ##########################################
    toPlot = Stations.Ploteo(df.iloc[step], Decode)
##############################################
    fig.add_trace(
        go.Scattermapbox(
        lat=toPlot.Latitude,
        lon=toPlot.Longitude,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=toPlot.Value, color= toPlot.Value
        ),
        text=toPlot.Address,
        visible = False,
        
    ))

# Make 10th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": Stations.Ploteo(df.iloc[i], Decode).Date}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]


fig.update_layout(
    autosize=True,
      title = dict( 
       
        x=0.5,
    ),
    hovermode='closest',
    width=500,
    height=700,
    margin=dict(
        l=100,
        r=100,
        b=200,
        t=100
        
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=float(toPlot.Latitude[0]),
            lon= float(toPlot.Longitude[0])
        ),
        pitch=0,
        zoom=10,
       
    ),
     sliders=sliders
)

# Use of clusters (Distrito, Barrio, cardinal, region)

In [30]:
StartDate = '2020-5-06 00:48'
EndDate = '2020-5-7 00:50'
Value='DockBikes'
Comparator = 'dow'
Accuracy = "h"
ClusterName= 'DistritoName'#DistritoName, CardinalName, RegionName
operation = 'mean'



df = Stations.ChooseInterval(data, StartDate, EndDate, Value)
df = Stations.ChooseAccuracy(df, Accuracy)
df= Stations.Clusterization(df, ClusterName, operation, Decode)

toPlot = Stations.Ploteo(df.iloc[20], Decode)
toPlot.getClusterMeans(df.iloc[20], Decode, ClusterName)

In [31]:
fig = go.Figure(go.Scattermapbox(
        lat=toPlot.Latitude,
        lon=toPlot.Longitude,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=toPlot.Value, color= toPlot.Value, showscale=True,colorscale= px.colors.sequential.Reds
        ),
        text=toPlot.Address,
       
        
    ))

fig.update_layout(
    title = dict( 
        text= str(toPlot.Date),
        x=0.5,
    ),

    autosize=True,
    hovermode='closest',
    width=750,
    height=500,
    margin=dict(
        l=50,
        r=0,
        b=50,
        t=50
        
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=float(toPlot.Latitude[0]),
            lon= float(toPlot.Longitude[0])
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()

# Grid de mapas

# Grid de mapas con slider